In [1]:
cd /Users/Liangqiaohao/Desktop/PV Lab/2020/March/Learning project/C2a

/Users/Liangqiaohao/Desktop/PV Lab/2020/March/Learning project/C2a


In [2]:
import numpy as np
import torch
import math
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import GPyOpt
import GPy

import os

import matplotlib as mpl
import matplotlib.tri as tri

import ternary
import pickle
import datetime

from collections import Counter
import matplotlib.ticker as ticker

import pyDOE
import random
from scipy.stats import norm

from sklearn.ensemble import RandomForestRegressor

In [3]:
dataset = pd.read_excel(r'/Users/Liangqiaohao/Desktop/PV Lab/2020/March/Learning project/C2a/C2a learning study.xlsx')
X = dataset.iloc[:,0:5]

X_run = X.groupby(['CsPbI', 'MAPbI', 'FAPbI'])['Instability index'].agg(lambda x: x.unique().mean())
X_run = (X_run.to_frame()).reset_index()

materials = ['CsPbI', 'MAPbI', 'FAPbI']
X_feature = X_run[['CsPbI', 'MAPbI', 'FAPbI']].values
y = np.array(X_run['Instability index'].values)

X_Cs = np.array(X_run['CsPbI'].values)
X_FA = np.array(X_run['FAPbI'].values)
X_MA = np.array(X_run['MAPbI'].values)

raw_mean = np.mean(y)
raw_std = np.std(y)

In [4]:
y = (y - raw_mean) / raw_std

In [5]:
# total 94 pts
n_initial = 2
n_test = int(math.ceil(len(y) * 0.30))
n_top = int(math.ceil(len(y) * 0.05))

top_indices = list(X_run.sort_values('Instability index').head(n_top).index)

# seed_list = [74,28,861,1526,2,151,19617,7,33,47302,66,552,671,25,1368,850,71,2148,2621,26, 6, 294, 9256, 851, 666]
# # 20

seed_list = [5, 892, 91, 2345, 391, 77, 223, 8258, 16, 3, 84,7646,6207,4530,8206,6260,2342,3112,5187,2737,8979,4018,8223,1678,9481,2232,
             461,7694,6091, 3189,2480,9106,1316,8500,2531,5329,9872,1388,8805,2726,119,3416,7747,3927,138,1488,232,4564,3492,9841]
# 50



In [6]:
def EI(X, model, y_best):
#     xi = 0.01
    X = X.reshape([1,X_feature.shape[1]])
    
    mean, std = model.predict(X)[0][0][0], model.predict(X)[1][0][0]

    z = (y_best - mean)/std
    return (y_best - mean) * norm.cdf(z) + std * norm.pdf(z)

def LCB(X, model):
    X = X.reshape([1,X_feature.shape[1]])
    
    mean, std = model.predict(X)[0][0][0], model.predict(X)[1][0][0]
    
    return 1 * std - 1 * mean

def MPI(X, model, y_best):
    X = X.reshape([1,X_feature.shape[1]])
    
    mean, std = model.predict(X)[0][0][0], model.predict(X)[1][0][0]
    
    z = (y_best - mean)/std
    return norm.cdf(z)

def calc_MAE(test_index, SL_model):
    SL_mae = 0
    
    for index in test_index:
        X_test = X_feature[index]
        y_test = y[index]
        
        X_test = X_test.reshape([1,X_feature.shape[1]])
        
        SL_mean, SL_std = SL_model.predict(X_test)[0][0][0], SL_model.predict(X_test)[1][0][0]
        
        sl_mae = (y_test - SL_mean)**2 / len(test_index)
        SL_mae += sl_mae       
    
    return SL_mae

def TopPercent(top_indices, index_):
    how_many = 0
    for i in index_:
        if i in top_indices:
            how_many += 1
    return how_many
 


In [7]:
ARD_ = False

Bias_kernel = GPy.kern.Bias(X_feature.shape[1], variance=1.)

Matern52_kernel = GPy.kern.Matern52(X_feature.shape[1], variance=1., ARD=ARD_) + Bias_kernel

In [8]:
index_collection = []
X_collection = []
y_collection = []
TopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    
    indices = list(np.arange(len(X_run)))
    
    
    X_top = []
    y_top = []
    for i in top_indices:
        X_top.append(X_feature[i])
        y_top.append(y[i])
        
    index_learn = indices.copy()
    
    index_ = random.sample(index_learn, n_initial)
    X_ = []
    y_ = []
    TopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        y_.append(y[i])
        index_learn.remove(i)
        
    index_mae = indices.copy()
    
    test_index = random.sample(index_mae, n_test)
    X_test = []
    y_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        y_test.append(y[i])
        index_mae.remove(i)
        
    X_full = []
    y_full = []    
    for i in index_mae:
        X_full.append(X_feature[i])
        y_full.append(y[i])
        
    
    
    assert len(indices) == len(X_full) + len(X_test)
    
#     GP_full = GPy.models.GPRegression(X = np.array(X_full), 
#                                       Y = np.array([[i] for i in y_full]), 
#                                       kernel= Matern52_kernel,
#                                       noise_var = 0.01
#                                       )

#     GP_full.optimize_restarts(num_restarts=10,
#                               parallel = True,
#                               robust = True,
#                               optimizer = 'bfgs',
#                               max_iters=100,
#                               verbose = False)
    
    for i in np.arange(len(index_learn)):
        
        y_best = np.min(y_)
        
        GP_learn = GPy.models.GPRegression(X = np.array(X_), 
                                           Y = np.array([[i] for i in y_]), 
                                           kernel= Matern52_kernel,
                                           noise_var = 0.01
                                          )

        GP_learn.optimize_restarts(num_restarts=10,
                                   parallel = True,
                                   robust = True,
                                   optimizer = 'bfgs',
                                   max_iters=100,
                                   verbose = False)
        
        next_index = None
        max_ac = -10**10
        for j in index_learn:
            X_j = X_feature[j]
            y_j = y[j]
            
            ac_value = EI(X_j, GP_learn, y_best)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        y_.append(y[next_index])
        index_learn.remove(next_index)
        index_.append(next_index)
        
        
        mae_ = calc_MAE(test_index, GP_learn)
        MAE_.append(mae_)
        TopPercent_.append(TopPercent(top_indices, index_))
        


    index_collection.append(index_)
    X_collection.append(X_)
    y_collection.append(y_)
    TopPercent_collection.append(TopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, y_collection, TopPercent_collection, MAE_collection])
np.save('STANDARD_GP_EI_master', master)
   

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed
initializing seed = 20
Finished seed
initializing seed = 21
Finished seed
initializing seed = 22
Finished seed
initializing seed = 23
Finished seed
initializing seed = 24
Finished seed
initializing seed = 25
Finished seed
initializing seed = 26
Finished seed
initializin

In [20]:
index_collection = []
X_collection = []
y_collection = []
TopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    
    indices = list(np.arange(len(X_run)))
    
    
    X_top = []
    y_top = []
    for i in top_indices:
        X_top.append(X_feature[i])
        y_top.append(y[i])
        
    index_learn = indices.copy()
    
    index_ = random.sample(index_learn, n_initial)
    X_ = []
    y_ = []
    TopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        y_.append(y[i])
        index_learn.remove(i)
        
    index_mae = indices.copy()
    
    test_index = random.sample(index_mae, n_test)
    X_test = []
    y_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        y_test.append(y[i])
        index_mae.remove(i)
        
    X_full = []
    y_full = []    
    for i in index_mae:
        X_full.append(X_feature[i])
        y_full.append(y[i])
        
    
    
    assert len(indices) == len(X_full) + len(X_test)
    
#     GP_full = GPy.models.GPRegression(X = np.array(X_full), 
#                                       Y = np.array([[i] for i in y_full]), 
#                                       kernel= Matern52_kernel,
#                                       noise_var = 0.01
#                                       )

#     GP_full.optimize_restarts(num_restarts=10,
#                               parallel = True,
#                               robust = True,
#                               optimizer = 'bfgs',
#                               max_iters=100,
#                               verbose = False)
    
    for i in np.arange(len(index_learn)):
        
        y_best = np.min(y_)
        
        GP_learn = GPy.models.GPRegression(X = np.array(X_), 
                                           Y = np.array([[i] for i in y_]), 
                                           kernel= Matern52_kernel,
                                           noise_var = 0.01
                                          )

        GP_learn.optimize_restarts(num_restarts=10,
                                   parallel = True,
                                   robust = True,
                                   optimizer = 'bfgs',
                                   max_iters=100,
                                   verbose = False)
        
        next_index = None
        max_ac = -10**10
        for j in index_learn:
            X_j = X_feature[j]
            y_j = y[j]
            
            ac_value = LCB(X_j, GP_learn)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        y_.append(y[next_index])
        index_learn.remove(next_index)
        index_.append(next_index)
        
        
        mae_ = calc_MAE(test_index, GP_learn)
        MAE_.append(mae_)
        TopPercent_.append(TopPercent(top_indices, index_))
        


    index_collection.append(index_)
    X_collection.append(X_)
    y_collection.append(y_)
    TopPercent_collection.append(TopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, y_collection, TopPercent_collection, MAE_collection])
np.save('STANDARD_GP_LCB11_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed
initializing seed = 20
Finished seed
initializing seed = 21
Finished seed
initializing seed = 22
Finished seed
initializing seed = 23
Finished seed
initializing seed = 24
Finished seed
initializing seed = 25
Finished seed
initializing seed = 26
Finished seed
initializin

In [19]:
# random sampling
index_collection = []
X_collection = []
y_collection = []
TopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    
    indices = list(np.arange(len(X_run)))
    
    
    X_top = []
    y_top = []
    for i in top_indices:
        X_top.append(X_feature[i])
        y_top.append(y[i])
        
    index_learn = indices.copy()
    
    index_ = random.sample(index_learn, n_initial)
    X_ = []
    y_ = []
    TopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        y_.append(y[i])
        index_learn.remove(i)
        
    index_mae = indices.copy()
    
    test_index = random.sample(index_mae, n_test)
    X_test = []
    y_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        y_test.append(y[i])
        index_mae.remove(i)
        
#     X_full = []
#     y_full = []    
#     for i in index_mae:
#         X_full.append(X_feature[i])
#         y_full.append(y[i])
        
    
    
#     assert len(indices) == len(X_full) + len(X_test)
    
#     GP_full = GPy.models.GPRegression(X = np.array(X_full), 
#                                       Y = np.array([[i] for i in y_full]), 
#                                       kernel= Matern52_kernel,
#                                       noise_var = 0.01
#                                       )

#     GP_full.optimize_restarts(num_restarts=10,
#                               parallel = True,
#                               robust = True,
#                               optimizer = 'bfgs',
#                               max_iters=100,
#                               verbose = False)
    
    for i in np.arange(len(index_learn)):
        
        GP_learn = GPy.models.GPRegression(X = np.array(X_), 
                                           Y = np.array([[i] for i in y_]), 
                                           kernel= Matern52_kernel,
                                           noise_var = 0.01
                                          )

        GP_learn.optimize_restarts(num_restarts=10,
                                   parallel = True,
                                   robust = True,
                                   optimizer = 'bfgs',
                                   max_iters=100,
                                   verbose = False)
        
        
        next_index = (random.sample(index_learn, 1))[0]
                
        X_.append(X_feature[next_index])
        y_.append(y[next_index])
        index_learn.remove(next_index)
        index_.append(next_index)
        
        
        mae_ = calc_MAE(test_index, GP_learn)
        MAE_.append(mae_)
        TopPercent_.append(TopPercent(top_indices, index_))
        


    index_collection.append(index_)
    X_collection.append(X_)
    y_collection.append(y_)
    TopPercent_collection.append(TopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, y_collection, TopPercent_collection, MAE_collection])
np.save('STANDARD_GP_RS_master', master)


initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed
initializing seed = 20
Finished seed
initializing seed = 21
Finished seed
initializing seed = 22
Finished seed
initializing seed = 23
Finished seed
initializing seed = 24
Finished seed


In [20]:
index_collection = []
X_collection = []
y_collection = []
TopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    
    indices = list(np.arange(len(X_run)))
    
    
    X_top = []
    y_top = []
    for i in top_indices:
        X_top.append(X_feature[i])
        y_top.append(y[i])
        
    index_learn = indices.copy()
    
    index_ = random.sample(index_learn, n_initial)
    X_ = []
    y_ = []
    TopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        y_.append(y[i])
        index_learn.remove(i)
        
    index_mae = indices.copy()
    
    test_index = random.sample(index_mae, n_test)
    X_test = []
    y_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        y_test.append(y[i])
        index_mae.remove(i)
        
    X_full = []
    y_full = []    
    for i in index_mae:
        X_full.append(X_feature[i])
        y_full.append(y[i])
        
    
    
    assert len(indices) == len(X_full) + len(X_test)
    
#     GP_full = GPy.models.GPRegression(X = np.array(X_full), 
#                                       Y = np.array([[i] for i in y_full]), 
#                                       kernel= Matern52_kernel,
#                                       noise_var = 0.01
#                                       )

#     GP_full.optimize_restarts(num_restarts=10,
#                               parallel = True,
#                               robust = True,
#                               optimizer = 'bfgs',
#                               max_iters=100,
#                               verbose = False)
    
    for i in np.arange(len(index_learn)):
        
        y_best = np.min(y_)
        
        GP_learn = GPy.models.GPRegression(X = np.array(X_), 
                                           Y = np.array([[i] for i in y_]), 
                                           kernel= Matern52_kernel,
                                           noise_var = 0.01
                                          )

        GP_learn.optimize_restarts(num_restarts=10,
                                   parallel = True,
                                   robust = True,
                                   optimizer = 'bfgs',
                                   max_iters=100,
                                   verbose = False)
        
        next_index = None
        max_ac = -10**10
        for j in index_learn:
            X_j = X_feature[j]
            y_j = y[j]
            
            ac_value = MPI(X_j, GP_learn, y_best)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        y_.append(y[next_index])
        index_learn.remove(next_index)
        index_.append(next_index)
        
        
        mae_ = calc_MAE(test_index, GP_learn)
        MAE_.append(mae_)
        TopPercent_.append(TopPercent(top_indices, index_))
        


    index_collection.append(index_)
    X_collection.append(X_)
    y_collection.append(y_)
    TopPercent_collection.append(TopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, y_collection, TopPercent_collection, MAE_collection])
np.save('STANDARD_GP_MPI_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed
initializing seed = 20
Finished seed
initializing seed = 21
Finished seed
initializing seed = 22
Finished seed
initializing seed = 23
Finished seed
initializing seed = 24
Finished seed


In [9]:
n_est = 10
# for 

In [10]:
def EI(X, RF_model, y_best):

    
    tree_predictions = []
    for j in np.arange(n_est):
        tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
    mean = np.mean(np.array(tree_predictions), axis=0)[0]

    
    std = np.std(np.array(tree_predictions), axis=0)[0]
    
    z = (y_best - mean)/std
    return (y_best - mean) * norm.cdf(z) + std * norm.pdf(z)

def LCB(X, RF_model):
    
    tree_predictions = []
    for j in np.arange(n_est):
        tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
    mean = np.mean(np.array(tree_predictions), axis=0)[0]

    
    std = np.std(np.array(tree_predictions), axis=0)[0]

    
    return 75 * std - 25 * mean

def MPI(X, RF_model, y_best):
    
    tree_predictions = []
    for j in np.arange(n_est):
        tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
    mean = np.mean(np.array(tree_predictions), axis=0)[0]
    
    std = np.std(np.array(tree_predictions), axis=0)[0]

    
    z = (y_best - mean)/std
    return norm.cdf(z)


def calc_MAE(test_index, RF_model):
    RF_mae = 0
    for index in test_index:
        X_test = X_feature[index]
        y_test = y[index]

        tree_predictions = []
        for j in np.arange(n_est):
            tree_predictions.append((RF_model.estimators_[j].predict(np.array([X_test]))).tolist())
        mean = np.mean(np.array(tree_predictions), axis=0)[0]

        std = np.std(np.array(tree_predictions), axis=0)[0]

        mae_ = (y_test - mean)**2 / len(test_index)
        RF_mae += mae_
        
    return RF_mae

In [11]:
index_collection = []
X_collection = []
y_collection = []
TopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    
    indices = list(np.arange(len(X_run)))
    
    
    X_top = []
    y_top = []
    for i in top_indices:
        X_top.append(X_feature[i])
        y_top.append(y[i])
        
    index_learn = indices.copy()
    
    index_ = random.sample(index_learn, n_initial)
    X_ = []
    y_ = []
    TopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        y_.append(y[i])
        index_learn.remove(i)
        
    index_mae = indices.copy()
    
    test_index = random.sample(index_mae, n_test)
    X_test = []
    y_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        y_test.append(y[i])
        index_mae.remove(i)
        
    X_full = []
    y_full = []    
    for i in index_mae:
        X_full.append(X_feature[i])
        y_full.append(y[i])
        
    
    
    assert len(indices) == len(X_full) + len(X_test)
    
#     GP_full = GPy.models.GPRegression(X = np.array(X_full), 
#                                       Y = np.array([[i] for i in y_full]), 
#                                       kernel= Matern52_kernel,
#                                       noise_var = 0.01
#                                       )

#     GP_full.optimize_restarts(num_restarts=10,
#                               parallel = True,
#                               robust = True,
#                               optimizer = 'bfgs',
#                               max_iters=100,
#                               verbose = False)
    
    for i in np.arange(len(index_learn)):
        
        y_best = np.min(y_)
        
        RF_model = RandomForestRegressor(n_estimators= n_est)
        RF_model.fit(X_, y_)
        
        next_index = None
        max_ac = -10**10
        for j in index_learn:
            X_j = X_feature[j]
            y_j = y[j]
            
            ac_value = EI(X_j, RF_model, y_best)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        y_.append(y[next_index])
        index_learn.remove(next_index)
        index_.append(next_index)
        
        
        mae_ = calc_MAE(test_index, RF_model)
        MAE_.append(mae_)
        TopPercent_.append(TopPercent(top_indices, index_))
        


    index_collection.append(index_)
    X_collection.append(X_)
    y_collection.append(y_)
    TopPercent_collection.append(TopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, y_collection, TopPercent_collection, MAE_collection])
np.save('STANDARD_RF_EI_master', master)


initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed
initializing seed = 20
Finished seed
initializing seed = 21
Finished seed
initializing seed = 22
Finished seed
initializing seed = 23
Finished seed
initializing seed = 24
Finished seed
initializing seed = 25
Finished seed
initializing seed = 26
Finished seed
initializin

In [28]:
index_collection = []
X_collection = []
y_collection = []
TopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    
    indices = list(np.arange(len(X_run)))
    
    
    X_top = []
    y_top = []
    for i in top_indices:
        X_top.append(X_feature[i])
        y_top.append(y[i])
        
    index_learn = indices.copy()
    
    index_ = random.sample(index_learn, n_initial)
    X_ = []
    y_ = []
    TopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        y_.append(y[i])
        index_learn.remove(i)
        
    index_mae = indices.copy()
    
    test_index = random.sample(index_mae, n_test)
    X_test = []
    y_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        y_test.append(y[i])
        index_mae.remove(i)
        
    X_full = []
    y_full = []    
    for i in index_mae:
        X_full.append(X_feature[i])
        y_full.append(y[i])
        
    
    
    assert len(indices) == len(X_full) + len(X_test)
    
#     GP_full = GPy.models.GPRegression(X = np.array(X_full), 
#                                       Y = np.array([[i] for i in y_full]), 
#                                       kernel= Matern52_kernel,
#                                       noise_var = 0.01
#                                       )

#     GP_full.optimize_restarts(num_restarts=10,
#                               parallel = True,
#                               robust = True,
#                               optimizer = 'bfgs',
#                               max_iters=100,
#                               verbose = False)
    
    for i in np.arange(len(index_learn)):
        
        y_best = np.min(y_)
        
        RF_model = RandomForestRegressor(n_estimators = n_est)
        RF_model.fit(X_, y_)
        
        next_index = None
        max_ac = -10**10
        for j in index_learn:
            X_j = X_feature[j]
            y_j = y[j]
            
            ac_value = LCB(X_j, RF_model)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        y_.append(y[next_index])
        index_learn.remove(next_index)
        index_.append(next_index)
        
        
        mae_ = calc_MAE(test_index, RF_model)
        MAE_.append(mae_)
        TopPercent_.append(TopPercent(top_indices, index_))
        


    index_collection.append(index_)
    X_collection.append(X_)
    y_collection.append(y_)
    TopPercent_collection.append(TopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, y_collection, TopPercent_collection, MAE_collection])
np.save('STANDARD_RF_LCB7525_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed
initializing seed = 20
Finished seed
initializing seed = 21
Finished seed
initializing seed = 22
Finished seed
initializing seed = 23
Finished seed
initializing seed = 24
Finished seed
initializing seed = 25
Finished seed
initializing seed = 26
Finished seed
initializin

In [15]:
index_collection = []
X_collection = []
y_collection = []
TopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    
    indices = list(np.arange(len(X_run)))
    
    
    X_top = []
    y_top = []
    for i in top_indices:
        X_top.append(X_feature[i])
        y_top.append(y[i])
        
    index_learn = indices.copy()
    
    index_ = random.sample(index_learn, n_initial)
    X_ = []
    y_ = []
    TopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        y_.append(y[i])
        index_learn.remove(i)
        
    index_mae = indices.copy()
    
    test_index = random.sample(index_mae, n_test)
    X_test = []
    y_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        y_test.append(y[i])
        index_mae.remove(i)
        
    X_full = []
    y_full = []    
    for i in index_mae:
        X_full.append(X_feature[i])
        y_full.append(y[i])
        
    
    
    assert len(indices) == len(X_full) + len(X_test)
    
#     GP_full = GPy.models.GPRegression(X = np.array(X_full), 
#                                       Y = np.array([[i] for i in y_full]), 
#                                       kernel= Matern52_kernel,
#                                       noise_var = 0.01
#                                       )

#     GP_full.optimize_restarts(num_restarts=10,
#                               parallel = True,
#                               robust = True,
#                               optimizer = 'bfgs',
#                               max_iters=100,
#                               verbose = False)
    
    for i in np.arange(len(index_learn)):
        
        y_best = np.min(y_)
        
        RF_model = RandomForestRegressor(n_estimators= n_est)
        RF_model.fit(X_, y_)
        
        next_index = None
        max_ac = -10**10
        for j in index_learn:
            X_j = X_feature[j]
            y_j = y[j]
            
            ac_value = MPI(X_j, RF_model, y_best)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        y_.append(y[next_index])
        index_learn.remove(next_index)
        index_.append(next_index)
        
        
        mae_ = calc_MAE(test_index, RF_model)
        MAE_.append(mae_)
        TopPercent_.append(TopPercent(top_indices, index_))
        


    index_collection.append(index_)
    X_collection.append(X_)
    y_collection.append(y_)
    TopPercent_collection.append(TopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, y_collection, TopPercent_collection, MAE_collection])
np.save('STANDARD_RF_MPI_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed
initializing seed = 20
Finished seed
initializing seed = 21
Finished seed
initializing seed = 22
Finished seed
initializing seed = 23
Finished seed
initializing seed = 24
Finished seed


In [16]:
index_collection = []
X_collection = []
y_collection = []
TopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    
    indices = list(np.arange(len(X_run)))
    
    
    X_top = []
    y_top = []
    for i in top_indices:
        X_top.append(X_feature[i])
        y_top.append(y[i])
        
    index_learn = indices.copy()
    
    index_ = random.sample(index_learn, n_initial)
    X_ = []
    y_ = []
    TopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        y_.append(y[i])
        index_learn.remove(i)
        
    index_mae = indices.copy()
    
    test_index = random.sample(index_mae, n_test)
    X_test = []
    y_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        y_test.append(y[i])
        index_mae.remove(i)
        
    X_full = []
    y_full = []    
    for i in index_mae:
        X_full.append(X_feature[i])
        y_full.append(y[i])
        
    
    
    assert len(indices) == len(X_full) + len(X_test)
    
#     GP_full = GPy.models.GPRegression(X = np.array(X_full), 
#                                       Y = np.array([[i] for i in y_full]), 
#                                       kernel= Matern52_kernel,
#                                       noise_var = 0.01
#                                       )

#     GP_full.optimize_restarts(num_restarts=10,
#                               parallel = True,
#                               robust = True,
#                               optimizer = 'bfgs',
#                               max_iters=100,
#                               verbose = False)
    
    for i in np.arange(len(index_learn)):
        
        y_best = np.min(y_)
        
        RF_model = RandomForestRegressor(n_estimators= n_est)
        RF_model.fit(X_, y_)
        
        next_index = (random.sample(index_learn, 1))[0]
                
        X_.append(X_feature[next_index])
        y_.append(y[next_index])
        index_learn.remove(next_index)
        index_.append(next_index)
        
        
        mae_ = calc_MAE(test_index, RF_model)
        MAE_.append(mae_)
        TopPercent_.append(TopPercent(top_indices, index_))
        


    index_collection.append(index_)
    X_collection.append(X_)
    y_collection.append(y_)
    TopPercent_collection.append(TopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, y_collection, TopPercent_collection, MAE_collection])
np.save('STANDARD_RF_RS_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed
initializing seed = 20
Finished seed
initializing seed = 21
Finished seed
initializing seed = 22
Finished seed
initializing seed = 23
Finished seed
initializing seed = 24
Finished seed


In [ ]:
# from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn.gaussian_process.kernels import (RBF, Matern, RationalQuadratic,
#                                               ExpSineSquared, DotProduct,
#                                               ConstantKernel)

# kernels = [1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 10.0)),
#            1.0 * RationalQuadratic(length_scale=1.0, alpha=0.1),
#            1.0 * ExpSineSquared(length_scale=1.0, periodicity=3.0,length_scale_bounds=(0.1, 10.0), periodicity_bounds=(1.0, 10.0)),
#            ConstantKernel(0.1, (0.01, 10.0))* (DotProduct(sigma_0=1.0, sigma_0_bounds=(0.1, 10.0)) ** 2),
#            1.0 * Matern(length_scale=1.0, length_scale_bounds=(1e-1, 10.0), nu=1.5)]

# model = GaussianProcessRegressor(alpha = 1e-10, 
#                                  kernel = kernels[0],
#                                  optimizer = 'fmin_l_bfgs_b', 
#                                  n_restarts_optimizer = 1, 
#                                  random_state=0,
#                                  normalize_y=True)

# # feature order: ['CsPbI', 'MAPbI', 'FAPbI']
# model.fit(X_feature, X_instability)

# Why these models from sklearn don't work well, I'm confused...